## Phase 2: Model Conversion

This notebook converts the custom trained .h5 models into edge-optimized formats.

*   TensorFlow.js format (.json + .bin) for use in a Progressive Web App (PWA)
*   TensorFlow Lite format (.tflite) for use in mobile apps

This ensures models are ready for execution on-device, enabling real-time predictions without server calls.


**1. Setting up the environment**

In [2]:
!pip install -q tensorflow tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.35.1 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [12]:
import os
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

In [5]:
# define paths
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = "/content/drive/My Drive/Edge_AI_Project/models"
H5_PATH = os.path.join(BASE_PATH, "saved_models")
TFJS_PATH = os.path.join(BASE_PATH, "tfjs_models")
TFLITE_PATH = os.path.join(BASE_PATH, "tflite_models")
LOG_PATH = os.path.join(BASE_PATH, "conversion_logs")

# Clean up and recreate directories
import shutil

# Remove existing converted models to start fresh
if os.path.exists(TFJS_PATH):
    shutil.rmtree(TFJS_PATH)
if os.path.exists(TFLITE_PATH):
    shutil.rmtree(TFLITE_PATH)

os.makedirs(TFJS_PATH, exist_ok=True)
os.makedirs(TFLITE_PATH, exist_ok=True)
os.makedirs(LOG_PATH, exist_ok=True)

print("✅ Directories cleaned and recreated")

Mounted at /content/drive
✅ Directories cleaned and recreated


**2. Model Conversion Functions**

In [13]:
def convert_to_tfjs_simple(h5_model_path, output_dir, model_name):
    """
    Simple TensorFlow.js conversion without model reconstruction
    """
    print(f"\n🔄 Converting {model_name} to TensorFlow.js...")

    # Load original model
    model = load_model(h5_model_path)
    print(f"   Loaded model: {model.input_shape} -> {model.output_shape}")

    # Test the model works
    if model.input_shape[1:] == (224, 224, 3):
        test_input = np.random.random((1, 224, 224, 3)).astype(np.float32)
    elif model.input_shape[1:] == (48, 48, 1):
        test_input = np.random.random((1, 48, 48, 1)).astype(np.float32)
    else:
        raise ValueError(f"Unexpected input shape: {model.input_shape}")

    test_output = model.predict(test_input, verbose=0)
    print(f"   Model test successful - Output shape: {test_output.shape}")

    # Convert to TensorFlow.js with UPDATED API
    print(f"   Converting to TensorFlow.js format...")
    tfjs.converters.save_keras_model(model, output_dir)

    print(f"✅ Successfully converted {model_name} to TensorFlow.js")

    # Verify the conversion
    model_files = os.listdir(output_dir)
    print(f"   Generated files: {model_files}")

    return model

def convert_to_tflite_simple(h5_model_path, output_path, model_name):
    """
    Simple TensorFlow Lite conversion
    """
    print(f"\n🔄 Converting {model_name} to TensorFlow Lite...")

    model = load_model(h5_model_path)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Post-training quantization
    tflite_model = converter.convert()

    with open(output_path, 'wb') as f:
        f.write(tflite_model)

    print(f"✅ Successfully converted {model_name} to TensorFlow Lite")


**3. Model configuration**

In [14]:
models = {
    "age": {
        "filename": "age_model.h5",
        "input_shape": (224, 224, 3)
    },
    "gender": {
        "filename": "gender_model.h5",
        "input_shape": (224, 224, 3)
    },
    "emotion": {
        "filename": "emotion_model.h5",
        "input_shape": (48, 48, 1)
    }
}

**4. Conversion and Logging Loop**

In [15]:
log_data = []

print("Starting model conversion process...\n")

for name, config in models.items():
    print(f"{'='*60}")
    print(f"🔄 Processing {name.upper()} model...")
    print(f"{'='*60}")

    h5_path = os.path.join(H5_PATH, config["filename"])
    tfjs_dir = os.path.join(TFJS_PATH, name)
    tflite_path = os.path.join(TFLITE_PATH, f"{name}.tflite")

    try:
        # Convert to TensorFlow.js
        model = convert_to_tfjs_simple(h5_path, tfjs_dir, name)

        # Convert to TFLite
        convert_to_tflite_simple(h5_path, tflite_path, name)

        # Calculate file sizes
        tfjs_size = sum(os.path.getsize(os.path.join(tfjs_dir, f)) for f in os.listdir(tfjs_dir))
        tflite_size = os.path.getsize(tflite_path)
        h5_size = os.path.getsize(h5_path)

        # Log the results
        log_data.append({
            "Model": name.capitalize(),
            "Input Shape": config["input_shape"],
            ".h5 Size (KB)": round(h5_size / 1024, 2),
            "TF.js Size (KB)": round(tfjs_size / 1024, 2),
            ".tflite Size (KB)": round(tflite_size / 1024, 2),
            "Model Layers": len(model.layers),
            "Output .tflite": tflite_path,
            "Output TF.js Dir": tfjs_dir
        })

        print(f"✅ {name.upper()} model conversion completed successfully!\n")

    except Exception as e:
        print(f"❌ Error processing {name} model: {str(e)}")
        import traceback
        traceback.print_exc()

print("\n" + "="*70)
print("Model conversion completed!")
print("="*70)

Starting model conversion process...

🔄 Processing AGE model...

🔄 Converting age to TensorFlow.js...


   Loaded model: (None, 224, 224, 3) -> (None, 1)


   Model test successful - Output shape: (1, 1)
   Converting to TensorFlow.js format...
failed to lookup keras version from the file,
    this is likely a weight only file
✅ Successfully converted age to TensorFlow.js
   Generated files: ['group1-shard1of3.bin', 'group1-shard2of3.bin', 'group1-shard3of3.bin', 'model.json']

🔄 Converting age to TensorFlow Lite...


Saved artifact at '/tmp/tmp48x_5ihp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132901482781520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482778640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482776720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482777104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482779600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482777296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482778064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482777680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482778256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482779024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482781328

   Loaded model: (None, 224, 224, 3) -> (None, 1)


   Model test successful - Output shape: (1, 1)
   Converting to TensorFlow.js format...
failed to lookup keras version from the file,
    this is likely a weight only file
✅ Successfully converted gender to TensorFlow.js
   Generated files: ['group1-shard1of3.bin', 'group1-shard2of3.bin', 'group1-shard3of3.bin', 'model.json']

🔄 Converting gender to TensorFlow Lite...


Saved artifact at '/tmp/tmpykvnstfb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132901498631760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498629840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498628496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498630032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498631184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498626576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498629264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498629648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498629456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498630608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901498632720

✅ Successfully converted gender to TensorFlow Lite
✅ GENDER model conversion completed successfully!

🔄 Processing EMOTION model...

🔄 Converting emotion to TensorFlow.js...
   Loaded model: (None, 48, 48, 1) -> (None, 1)


   Model test successful - Output shape: (1, 1)
   Converting to TensorFlow.js format...
failed to lookup keras version from the file,
    this is likely a weight only file
✅ Successfully converted emotion to TensorFlow.js
   Generated files: ['group1-shard1of3.bin', 'group1-shard2of3.bin', 'group1-shard3of3.bin', 'model.json']

🔄 Converting emotion to TensorFlow Lite...


Saved artifact at '/tmp/tmpgiaayh1g'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132901706067280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706069584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706069776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706068048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706066896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706067856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706056144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706068816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706065936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706070928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901706064400

**5. Save conversion logs**

In [17]:
if log_data:
    df = pd.DataFrame(log_data)
    log_file = os.path.join(LOG_PATH, "simple_conversion_summary.csv")
    df.to_csv(log_file, index=False)

    print(f"\n📊 Conversion Summary:")
    print(df.to_string(index=False))
    print(f"\n💾 Log saved to: {log_file}")




📊 Conversion Summary:
  Model   Input Shape  .h5 Size (KB)  TF.js Size (KB)  .tflite Size (KB)  Model Layers                                                              Output .tflite                                                   Output TF.js Dir
    Age (224, 224, 3)       26885.20          8979.97            2443.16           156     /content/drive/My Drive/Edge_AI_Project/models/tflite_models/age.tflite     /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/age
 Gender (224, 224, 3)       26885.20          8979.97            2443.16           156  /content/drive/My Drive/Edge_AI_Project/models/tflite_models/gender.tflite  /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/gender
Emotion   (48, 48, 1)       25666.21          8548.44            2142.49             9 /content/drive/My Drive/Edge_AI_Project/models/tflite_models/emotion.tflite /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/emotion

💾 Log saved to: /content/drive/My Drive/Edge_AI_Project/

**6. Verification**

In [18]:
print(f"\n Verification:")
print("Checking if model.json files were created correctly...")

for name in models.keys():
    tfjs_dir = os.path.join(TFJS_PATH, name)
    model_json_path = os.path.join(tfjs_dir, "model.json")

    if os.path.exists(model_json_path):
        print(f"   ✅ {name}: model.json exists")

        # Check file size
        json_size = os.path.getsize(model_json_path)
        if json_size > 1000:  # Should be at least 1KB
            print(f"   ✅ {name}: model.json has reasonable size ({json_size} bytes)")
        else:
            print(f"   ⚠️  {name}: model.json seems too small ({json_size} bytes)")
    else:
        print(f"   ❌ {name}: model.json not found")

print(f"\n Verification CompleteE!")


 Verification:
Checking if model.json files were created correctly...
   ✅ age: model.json exists
   ✅ age: model.json has reasonable size (158430 bytes)
   ✅ gender: model.json exists
   ✅ gender: model.json has reasonable size (158430 bytes)
   ✅ emotion: model.json exists
   ✅ emotion: model.json has reasonable size (9148 bytes)

 Verification CompleteE!
